## Mover un grupo de volúmen a otro sistema

Puede mover un grupo de volúmemes LVM completo a otro sistema. Se recomienda que se utilice los comandos vgexport y vgimport cuando haga esto.

El comando **vgexport** hace que un grupo de volumenes inactivo sea inaccesible para el sistema, lo que permite separar los volumenes físicos para poder manipularlos y entonces con el comando **vgimport** se hace que el grupo de volumenes sea accesible nuevamente para despues de que el comando **vgexport** lo haya desactivado, es como un complemento de como reemplazar discos.

#### Usando el laboratorio 11 concluido

Para mover el grupo de volúmenes de un sistema a otro hay que realizar algunos pasos.

Primero comprobamos que tengamos el grupo de volúmenes

`vgs`

***
```
root@debian12lvm1:~# vgs
  VG    #PV #LV #SN Attr   VSize  VFree
  newvg   5   4   0 wz--n- 24.98g 6.98g
```
***

Ahora vemos los volúmenes físicos

`pvs`

***
```
root@debian12lvm1:~# pvs
  PV         VG    Fmt  Attr PSize  PFree   
  /dev/sdb1  newvg lvm2 a--  <5.00g   <5.00g
  /dev/sdc1  newvg lvm2 a--  <5.00g 1020.00m
  /dev/sdd1  newvg lvm2 a--  <5.00g       0 
  /dev/sde1  newvg lvm2 a--  <5.00g       0 
  /dev/sdf1  newvg lvm2 a--  <5.00g 1012.00m

```
***

Ahora hay que asegurarse de que ningun usuario accesa a los archivos en los volúmenes activos en el grupos de volúmenes y luego desmontar el volúmen lógico

Comprobamos que el volúmen lógico este desmontado 

`df -h`

***
```
root@debian12lvm1:~# df -h
S.ficheros     Tamaño Usados  Disp Uso% Montado en
udev             960M      0  960M   0% /dev
tmpfs            197M   924K  196M   1% /run
/dev/sda1        8.9G   1.9G  6.5G  23% /
tmpfs            984M      0  984M   0% /dev/shm
tmpfs            5.0M      0  5.0M   0% /run/lock
tmpfs            197M      0  197M   0% /run/user/1000
```
***

no aparece el punto de montaje a **/mnt** de **/dev/newvg/newvl**

Ahora desactvamos el grupo de volumenes

`vgchange -a n newvg`

***
```
root@debian12lvm1:~# vgchange -a n newvg 
  0 logical volume(s) in volume group "newvg" now active
```
***

Ahora procedemos a exportar 

`newvg`

***
```
root@debian12lvm1:~# vgexport newvg 
  Volume group "newvg" successfully exported
```
***


Ahora ejecutamos


`pvscan`

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdc1    is in exported VG newvg [<5.00 GiB / 1020.00 MiB free]
  PV /dev/sdd1    is in exported VG newvg [<5.00 GiB / 0    free]
  PV /dev/sde1    is in exported VG newvg [<5.00 GiB / 0    free]
  PV /dev/sdf1    is in exported VG newvg [<5.00 GiB / 1012.00 MiB free]
  PV /dev/sdb1    is in exported VG newvg [<5.00 GiB / <5.00 GiB free]
  Total: 5 [24.98 GiB] / in use: 5 [24.98 GiB] / in no VG: 0 [0   ]


```
***

Ahora, si físicamente tuviesemos identificados los discos, los podríamos extraer y llevarnoslos a otro sistema, los podríamos conectar, encender el servidor y despues ejecutar 


`vgimport newvg`

***
```
root@debian12lvm1:~# vgimport newvg 
  Volume group "newvg" successfully imported
```
***

Ahora ejecutamos


`pvscan`

***
```
root@debian12lvm1:~# pvscan
  PV /dev/sdc1   VG newvg           lvm2 [<5.00 GiB / 1020.00 MiB free]
  PV /dev/sdd1   VG newvg           lvm2 [<5.00 GiB / 0    free]
  PV /dev/sde1   VG newvg           lvm2 [<5.00 GiB / 0    free]
  PV /dev/sdf1   VG newvg           lvm2 [<5.00 GiB / 1012.00 MiB free]
  PV /dev/sdb1   VG newvg           lvm2 [<5.00 GiB / <5.00 GiB free]
  Total: 5 [24.98 GiB] / in use: 5 [24.98 GiB] / in no VG: 0 [0   ]

```
***

y ahora activar el volúmen de grupo

``

***
```
root@debian12lvm1:~# vgchange -a y newvg 
  4 logical volume(s) in volume group "newvg" now active
```
***

